# Solution 4

In [3]:
import os
import numpy as np
import pandas as pd

import os
from castle.common.priori_knowledge import PrioriKnowledge
# os.path.abspath('.')

In [4]:
import plotly.express as px
import plotly.graph_objects as go


In [5]:
from methods import *

# Read Data

In [6]:
dataset = 'dataset_1'

def read_data(dataname = 'dataset_1'):
    # alarm data
    alarms = pd.read_csv(r'./datasets/dataset_1/alarm.csv')
    # causal_prior
    causal_prior= np.load(r'./datasets/dataset_1/causal_prior.npy')

    # topology
    topology = np.load(r'./datasets/dataset_1/topology.npy')

    # rca
    rca_prior = pd.read_csv(r'./datasets/dataset_1/rca_prior.csv')

    print(f"shape of alarm data: {alarms.shape}")
    print(f"shape of causal prior matrix: {causal_prior.shape}")
    print(f"shape of topology prior matrix: {topology.shape}")
    print(f"shape of rca prior matrix: {rca_prior.shape}")
    # Notes: topology.npy and rca_prior.csv are not used in this script.
    
    return alarms, causal_prior, topology, rca_prior

In [7]:
# dataset = 'dataset_4'

def read_data(dataset = 'dataset_1'):

    # alarm data
    alarms = pd.read_csv(r'./datasets/{}/alarm.csv'.format(dataset))
    # causal_prior
    causal_prior= np.load(r'./datasets/{}/causal_prior.npy'.format(dataset))

    # # topology
    topology = np.load(r'./datasets/{}/topology.npy'.format(dataset))

    # # rca
    rca_prior = pd.read_csv(r'./datasets/{}/rca_prior.csv'.format(dataset))

    print(f"shape of alarm data: {alarms.shape}")
    print(f"shape of causal prior matrix: {causal_prior.shape}")
    print(f"shape of topology prior matrix: {topology.shape}")
    print(f"shape of rca prior matrix: {rca_prior.shape}")
    print("_________")
    
    prior_knowledge = PrioriKnowledge(causal_prior.shape[0])
    for i, j in zip(*np.where(causal_prior == 1)):
        prior_knowledge.add_required_edge(i, j)

    for i, j in zip(*np.where(causal_prior == 0)):
        prior_knowledge.add_forbidden_edge(i, j)
    
    return alarms, causal_prior, topology, rca_prior, prior_knowledge


# Notes: topology.npy and rca_prior.csv are not used in this script.

In [8]:
list_of_dataset =[]
for dataset in ['dataset_1', 'dataset_2', 'dataset_3']:
    alarms, causal_prior, topology, rca_prior, prior_knowledge = read_data(dataset = dataset)
    # print(f"dataset: {dataset}")
    # print(f"shape of alarm data: {alarms.shape}")
    # print(f"shape of causal prior matrix: {causal_prior.shape}")
    # print(f"shape of topology prior matrix: {topology.shape}")
    # print(f"shape of rca prior matrix: {rca_prior.shape}")
    # print("_________")
    
    list_of_dataset.append([dataset,alarms, causal_prior, topology, rca_prior, prior_knowledge])
    
# alarms, causal_prior, topology, rca_prior, prior_knowledge = read_data(dataset = 'dataset_1')
# alarms, causal_prior, topology, rca_prior, prior_knowledge = read_data(dataset = 'dataset_2')
# alarms, causal_prior, topology, rca_prior, prior_knowledge= read_data(dataset = 'dataset_3')
# alarms, causal_prior, topology, rca_prior = read_data(dataname = 'dataset_1')

shape of alarm data: (141853, 4)
shape of causal prior matrix: (39, 39)
shape of topology prior matrix: (35, 35)
shape of rca prior matrix: (49, 2)
_________
shape of alarm data: (132688, 4)
shape of causal prior matrix: (49, 49)
shape of topology prior matrix: (42, 42)
shape of rca prior matrix: (47, 2)
_________
shape of alarm data: (198962, 4)
shape of causal prior matrix: (31, 31)
shape of topology prior matrix: (39, 39)
shape of rca prior matrix: (43, 2)
_________


In [22]:
list_of_dataset

[['dataset_1',
          alarm_id  device_id  start_timestamp  end_timestamp
  0             12         25                5             38
  1             10         25                7             45
  2              2         24               13             53
  3             35         17               13             86
  4             35         32               13             14
  ...          ...        ...              ...            ...
  141848        25         21           604772         604883
  141849        25          2           604779         604863
  141850        25         26           604787         604827
  141851        27          0           604787         604881
  141852        27         26           604816         604856
  
  [141853 rows x 4 columns],
  array([[-1., -1., -1., ...,  0., -1., -1.],
         [-1., -1., -1., ...,  0., -1., -1.],
         [-1., -1., -1., ...,  0., -1., -1.],
         ...,
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [-

# 从预处理过的数据中读取

In [23]:
pre_solution = '01_topo_only_winsize_500'

list_of_dataset =[]
for dataset in ['dataset_1', 'dataset_2', 'dataset_3']:
    alarms, causal_prior, topology, rca_prior, prior_knowledge = read_data(dataset = dataset)
    samples = pd.read_csv(r'./datasets_processed/{}/{}.csv'.format(pre_solution,dataset))
    # print(f"dataset: {dataset}")
    # print(f"shape of alarm data: {alarms.shape}")
    # print(f"shape of causal prior matrix: {causal_prior.shape}")
    # print(f"shape of topology prior matrix: {topology.shape}")
    # print(f"shape of rca prior matrix: {rca_prior.shape}")
    # print("_________")
    
    list_of_dataset.append([dataset,alarms, causal_prior, topology, rca_prior, prior_knowledge,samples])

shape of alarm data: (141853, 4)
shape of causal prior matrix: (39, 39)
shape of topology prior matrix: (35, 35)
shape of rca prior matrix: (49, 2)
_________
shape of alarm data: (132688, 4)
shape of causal prior matrix: (49, 49)
shape of topology prior matrix: (42, 42)
shape of rca prior matrix: (47, 2)
_________
shape of alarm data: (198962, 4)
shape of causal prior matrix: (31, 31)
shape of topology prior matrix: (39, 39)
shape of rca prior matrix: (43, 2)
_________


In [24]:
len(list_of_dataset), len(list_of_dataset[0])

(3, 7)

# 建模

In [33]:
import importlib

import methods
importlib.reload(methods)
from methods import Causal
from methods import causal_all

In [34]:
causal_all(list_of_dataset = list_of_dataset, preprocessed = False)

All dataset processed
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Get kwargs in causal pipeline:  

2023-10-01 22:28:56,706 - /Users/zhitaogao/opt/anaconda3/envs/gcastle/lib/python3.9/site-packages/castle/backend/__init__.py[line:36] - INFO: You can use `os.environ['CASTLE_BACKEND'] = backend` to set the backend(`pytorch` or `mindspore`).
2023-10-01 22:28:56,795 - /Users/zhitaogao/opt/anaconda3/envs/gcastle/lib/python3.9/site-packages/castle/algorithms/__init__.py[line:36] - INFO: You are using ``pytorch`` as the backend.


{'data': ['dataset_1',         alarm_id  device_id  start_timestamp  end_timestamp
0             12         25                5             38
1             10         25                7             45
2              2         24               13             53
3             35         17               13             86
4             35         32               13             14
...          ...        ...              ...            ...
141848        25         21           604772         604883
141849        25          2           604779         604863
141850        25         26           604787         604827
141851        27          0           604787         604881
141852        27         26           604816         604856

[141853 rows x 4 columns], array([[-1., -1., -1., ...,  0., -1., -1.],
       [-1., -1., -1., ...,  0., -1., -1.],
       [-1., -1., -1., ...,  0., -1., -1.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [-1., -1., -1., ...,  0., -1., -1.

2023-10-01 22:28:59,786 - /Users/zhitaogao/opt/anaconda3/envs/gcastle/lib/python3.9/site-packages/castle/backend/__init__.py[line:36] - INFO: You can use `os.environ['CASTLE_BACKEND'] = backend` to set the backend(`pytorch` or `mindspore`).
2023-10-01 22:28:59,832 - /Users/zhitaogao/opt/anaconda3/envs/gcastle/lib/python3.9/site-packages/castle/algorithms/__init__.py[line:36] - INFO: You are using ``pytorch`` as the backend.
2023-10-01 22:28:59,903 - /Users/zhitaogao/opt/anaconda3/envs/gcastle/lib/python3.9/site-packages/castle/algorithms/gradient/notears/linear.py[line:195] - INFO: [start]: n=34176, d=39, iter_=100, h_=1e-08, rho_=1e+16
/Users/zhitaogao/opt/anaconda3/envs/gcastle/lib/python3.9/site-packages/castle/algorithms/gradient/notears/linear.py:144: RuntimeWarning: invalid value encountered in logaddexp
  loss = 1.0 / X.shape[0] * (np.logaddexp(0, M) - X * M).sum()


Window size 1500 Total 62211 removed 3905 alarms
Window size 1500 Total 58306 alarms
Method notears Start fitting dataset_1 Shape of samples: (34176, 39)
{'data': ['dataset_2',         alarm_id  device_id  start_timestamp  end_timestamp
0             45         20                6             43
1             27         23               11             40
2              6         24               13             60
3             15         26               17             32
4             23         40               18             68
...          ...        ...              ...            ...
132683        38         25           604716         604725
132684        33         20           604718         604725
132685        17          9           604720         604800
132686        17          4           604757         604796
132687        45         22           604785         604800

[132688 rows x 4 columns], array([[-1., -1., -1., ..., -1., -1., -1.],
       [-1., -1., -1., ..., -1.

...

Window size 1500 Total 132010 removed 9632 alarms
Window size 1500 Total 122378 alarms
Looking for connected, Processing device: 7 Number of Connected: 31
Number of alarms connected: 152070 Percentage 0.76
Window size 1000 Total 152070 removed 13851 alarms
Window size 1000 Total 138219 alarms
Window size 1500 Total 152070 removed 11793 alarms
Window size 1500 Total 140277 alarms
Looking for connected, Processing device: 8 Number of Connected: 24
Number of alarms connected: 130289 Percentage 0.65
Window size 1000 Total 130289 removed 11734 alarms
Window size 1000 Total 118555 alarms
Window size 1500 Total 130289 removed 10009 alarms
Window size 1500 Total 120280 alarms
Looking for connected, Processing device: 9 Number of Connected: 19
Number of alarms connected: 80880 Percentage 0.41
Window size 1000 Total 80880 removed 7451 alarms
Window size 1000 Total 73429 alarms
Window size 1500 Total 80880 removed 6362 alarms
Window size 1500 Total 74518 alarms
Looking for connected, Processing d

2023-10-01 22:29:07,068 - /Users/zhitaogao/opt/anaconda3/envs/gcastle/lib/python3.9/site-packages/castle/algorithms/gradient/notears/linear.py[line:206] - INFO: [iter 0] h=0.000e+00, loss=nan, rho=1.0e+00
2023-10-01 22:29:07,069 - /Users/zhitaogao/opt/anaconda3/envs/gcastle/lib/python3.9/site-packages/castle/algorithms/gradient/notears/linear.py[line:222] - INFO: FINISHED


Window size 1000 Total 111416 removed 9669 alarms
Window size 1000 Total 101747 alarms
Window size 1500 Total 111416 removed 8310 alarms
Window size 1500 Total 103106 alarms
Looking for connected, Processing device: 28 Number of Connected: 18
Number of alarms connected: 84681 Percentage 0.43
Window size 1000 Total 84681 removed 7521 alarms
Window size 1000 Total 77160 alarms
Window size 1500 Total 84681 removed 6515 alarms
Window size 1500 Total 78166 alarms
Looking for connected, Processing device: 29 Number of Connected: 24
Number of alarms connected: 122425 Percentage 0.62
Window size 1000 Total 122425 removed 10444 alarms
Window size 1000 Total 111981 alarms
Window size 1500 Total 122425 removed 8925 alarms
Window size 1500 Total 113500 alarms
Looking for connected, Processing device: 30 Number of Connected: 30
Number of alarms connected: 148908 Percentage 0.75
Window size 1000 Total 148908 removed 12972 alarms
Window size 1000 Total 135936 alarms
Window size 1500 Total 148908 remo

2023-10-01 22:29:08,943 - /Users/zhitaogao/opt/anaconda3/envs/gcastle/lib/python3.9/site-packages/castle/algorithms/gradient/notears/linear.py[line:195] - INFO: [start]: n=38100, d=31, iter_=100, h_=1e-08, rho_=1e+16
/Users/zhitaogao/opt/anaconda3/envs/gcastle/lib/python3.9/site-packages/castle/algorithms/gradient/notears/linear.py:144: RuntimeWarning: invalid value encountered in logaddexp
  loss = 1.0 / X.shape[0] * (np.logaddexp(0, M) - X * M).sum()
2023-10-01 22:29:10,599 - /Users/zhitaogao/opt/anaconda3/envs/gcastle/lib/python3.9/site-packages/castle/algorithms/gradient/notears/linear.py[line:206] - INFO: [iter 0] h=0.000e+00, loss=nan, rho=1.0e+00
2023-10-01 22:29:10,599 - /Users/zhitaogao/opt/anaconda3/envs/gcastle/lib/python3.9/site-packages/castle/algorithms/gradient/notears/linear.py[line:222] - INFO: FINISHED


shape:  (31, 31)
Saved solution 20231001-222910-2 for dataset dataset_3 to ./submission/solution20231001-222910-2/dataset_3_graph_matrix.npy
Saved solution 2 for dataset 1 to ./submission/solution2/1_graph_matrix.npy
All dataset processed


# Evaluation

In [20]:
from castle.common import GraphDAG

In [15]:
# read solution

s_name = 'solution1'
graph_matrix_1 = np.load(r'./submission/{}/dataset_1_graph_matrix.npy'.format(s_name))
graph_matrix_2 = np.load(r'./submission/{}/dataset_2_graph_matrix.npy'.format(s_name))
graph_matrix_3 = np.load(r'./submission/{}/dataset_3_graph_matrix.npy'.format(s_name))

In [16]:
GraphDAG(graph_matrix_1,graph_matrix_2)


IndexError: index 39 is out of bounds for axis 0 with size 39